In [1]:
# import packages to get the relative path starting from this notebook
import sys, os
this_notebook_path = os.path.dirname(os.path.realpath("__file__"))
sys.path.append(os.path.join(this_notebook_path,"..","src"))

In [2]:
from server import Server, Experiment
from instrument import VirtualInstrument, FloatProperty

class Voltmeter(VirtualInstrument):
    def __init__(self):
        super().__init__("Voltmeter")
        self.voltage = FloatProperty('V', readonly=True)
        self.add_command(":voltage", self.voltage)

class CurrentSource(VirtualInstrument):
    def __init__(self):
        super().__init__("Current source")
        self.current = FloatProperty('A')
        self.add_command(":current", self.current)

class OhmExperiment(Experiment):
    ports = (PORT_V := 9001,
             PORT_I := 9002)

    def __init__(self, client_ip, resistance):
        super().__init__(client_ip)
        self.resistance = resistance
        self.instruments = {
            self.PORT_V: Voltmeter(),
            self.PORT_I: CurrentSource()
            }
        self.instruments[self.PORT_I].current.add_set_hook(self.sync)
                
    def sync(self, *args):
        self.instruments[self.PORT_V].voltage.value = \
            self.instruments[self.PORT_I].current.value * self.resistance

if __name__ == "__main__":

    server = Server(OhmExperiment, local=True, resistance=1e3)
    try:
        server.run()
    except Exception:
        server.shutdown_event.set()
        raise

Socket bound at localhost:9001
Socket bound at localhost:9002
Listening at ('127.0.0.1', 9001)
Listening at ('127.0.0.1', 9002)
Connection from ('127.0.0.1', 55335)
Connection from ('127.0.0.1', 55336)


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Git_Repositories\scpi_virtual\jupyter_notebooks\..\src\server.py", line 45, in run
    instrument.process_messages()
  File "C:\Git_Repositories\scpi_virtual\jupyter_notebooks\..\src\instrument.py", line 156, in process_messages
    self.oqueue.put(";".join(responses).encode("latin1") + b"\n")
TypeError: sequence item 0: expected str instance, NoneType found


[[':VOLT', True]]
[[':CURR', False, ['3']]]
